In [ ]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
model = Network('model')

node = np.empty(4, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Source(model,'Source')
node[3] = Sink(model,'Sink')

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 2, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

node[0].setService(jobclass[0], Erlang(3,2))
node[0].setService(jobclass[1], HyperExp(0.5,3,10))

node[1].setService(jobclass[0], HyperExp(0.1,1,10))
node[1].setService(jobclass[1], Exp(1))

node[2].setArrival(jobclass[1], Exp(0.1))

K = model.getNumberOfClasses()

P = model.initRoutingMatrix()
pmatrix = np.empty(K, dtype=object)
pmatrix[0] = [[0,1,0,0],[1.0,0,0,0], [0,0,0,0], [0,0,0,0]]
pmatrix[1] = [[0,1,0,0], [0,0,0,1.0], [1.0,0,0,0], [0,0,0,0]]
P.setRoutingMatrix(jobclass, node, pmatrix)
            
model.link(P);


In [ ]:
solver = np.array([], dtype=object)
#solver = np.append(solver, SolverCTMC(model,'cutoff',3))
solver = np.append(solver, SolverJMT(model,'seed',23000,'verbose',True,'keep',True))
solver = np.append(solver, SolverSSA(model,'seed',23000,'verbose',False,'samples',50000))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model,'exact'))
solver = np.append(solver, SolverMAM(model))
#np.append(solver, LINE(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()
